<a href="https://colab.research.google.com/github/GabrielMou21/projeto-bd-colab/blob/main/Projeto_BD_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados em memória
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

# Criar tabelas
cursor.execute('''
    CREATE TABLE IF NOT EXISTS clientes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        email TEXT UNIQUE,
        data_cadastro DATE DEFAULT CURRENT_DATE
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        categoria TEXT,
        preco REAL CHECK(preco > 0),
        estoque INTEGER DEFAULT 0
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS pedidos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        cliente_id INTEGER,
        data_pedido TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        status TEXT DEFAULT 'Pendente',
        FOREIGN KEY (cliente_id) REFERENCES clientes(id) ON DELETE CASCADE
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS itens_pedido (
        pedido_id INTEGER,
        produto_id INTEGER,
        quantidade INTEGER CHECK(quantidade > 0),
        preco_unitario REAL,
        PRIMARY KEY (pedido_id, produto_id),
        FOREIGN KEY (pedido_id) REFERENCES pedidos(id) ON DELETE CASCADE,
        FOREIGN KEY (produto_id) REFERENCES produtos(id)
    );
''')

# Inserir dados
cursor.executemany("INSERT INTO clientes (nome, email) VALUES (?, ?)", [
    ('Ana Silva', 'ana@email.com'),
    ('Carlos Oliveira', 'carlos@email.com'),
    ('Mariana Souza', 'mariana@email.com'),
    ('João Santos', 'joao@email.com')
])

cursor.executemany("INSERT INTO produtos (nome, categoria, preco, estoque) VALUES (?, ?, ?, ?)", [
    ('Notebook Dell', 'Eletrônicos', 4500.00, 10),
    ('Smartphone Samsung', 'Eletrônicos', 2200.00, 15),
    ('Mesa de Escritório', 'Móveis', 800.00, 5),
    ('Cadeira Ergonômica', 'Móveis', 650.00, 8),
    ('Mouse Sem Fio', 'Acessórios', 120.00, 20)
])

cursor.executemany("INSERT INTO pedidos (cliente_id, status) VALUES (?, ?)", [
    (1, 'Pendente'),
    (2, 'Processando'),
    (3, 'Pendente')
])

cursor.executemany("INSERT INTO itens_pedido VALUES (?, ?, ?, ?)", [
    (1, 1, 1, 4500.00),
    (1, 5, 2, 120.00),
    (2, 2, 1, 2200.00),
    (3, 3, 1, 800.00),
    (3, 4, 2, 650.00)
])

# Consultar pedidos com detalhes
cursor.execute('''
    SELECT p.id AS Pedido, c.nome AS Cliente, pr.nome AS Produto, ip.quantidade AS Quantidade, ip.preco_unitario AS Preço
    FROM pedidos p
    JOIN clientes c ON p.cliente_id = c.id
    JOIN itens_pedido ip ON p.id = ip.pedido_id
    JOIN produtos pr ON ip.produto_id = pr.id
''')

df = pd.DataFrame(cursor.fetchall(), columns=['Pedido', 'Cliente', 'Produto', 'Quantidade', 'Preço'])
print(df)

# Fechar conexão
conn.close()
